# Sea level projection
### San Sebastian | Genova | La Rochelle | Valencia
Written by Math van Soest - Deltares 
17-08-'23

In [1]:

# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math
import itertools
import glob

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# import coclicodata functionalities (TODO: import as package when ETL is decoupled from CoCliCo STAC; Etiënne & Floris now whereabouts)
sys.path.append(str(cwd.parent.parent.parent.joinpath("coclicodata"))) # import functionality from local clone of coclicodata (make sure you pull the latest version)

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("11205479-coclico","FULLTRACK_DATA","WP3")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\soest\AppData\Local\Temp\ipykernel_19400\815791757.py:7: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir
dataset_dir_path = dataset_dir.joinpath("SLP_MvS.nc")
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

In [3]:
# write csv to netcdf

# open all csv files in different dirs
all_files = []
for dir in os.listdir(dataset_dir):
    if '.' not in dir: # arbitrary, no file extension to determine whether it is dir
        all_files.append(glob.glob(os.path.join(dataset_dir , dir,  "*.CSV")))

# read csv and convert to nc files
li = []
for filename in list(itertools.chain(*all_files)):
   
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

    ds = xr.Dataset.from_dataframe(df)
    ds.to_netcdf(filename.replace('.csv', '.nc'))

# make one dataframe
#df = pd.concat(li, axis=0, ignore_index=True)

# Convert the pandas dataframe to an xarray dataset
#ds = xr.Dataset.from_dataframe(df)

# Write the xarray dataset to a netCDF file
#ds.to_netcdf('C:\Users\soest\Documents\GitHub\coclicofiles\SLR_MvS.nc')

In [4]:
# open datasets (only first file, rest is the same)
ds = xr.open_dataset(list(itertools.chain(*all_files))[0].replace('.csv', '.nc'))

# check original dataset
ds

<xarray.Dataset>
Dimensions:  (index: 4)
Coordinates:
  * index    (index) int64 0 1 2 3
Data variables:
    years    (index) int64 ...
    low      (index) float64 ...
    median   (index) float64 ...
    high     (index) float64 ...

In [6]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=list(itertools.chain(*all_files))[0].replace('.csv', '.nc'), 
                 working_dir=CF_dir
                 )

# save original CF compliancy (for first file)
save_compliancy(cap, testfile=list(itertools.chain(*all_files))[0].replace('.csv', '.nc'), working_dir=CF_dir)                 

### Make CF compliant alterations to the NetCDF files (dataset dependent)


In [21]:
# open datasets
ds_list = []
for i in list(itertools.chain(*all_files)):
    ds_list.append(xr.open_dataset(i.replace('.csv', '.nc')))

In [22]:
import json

# NetCDF attribute alterations by means of metadata template
f_global = open(gca_data_dir.joinpath("metadata", "metadata_SLP.json"))
meta_global = json.load(f_global)

for i in ds_list:
    for attr_name, attr_val in meta_global.items():
        if attr_name == 'PROVIDERS':
            attr_val = json.dumps(attr_val)
        i.attrs[attr_name] = attr_val

    i.attrs['Conventions'] = "CF-1.8"

In [23]:
# NetCDF variable and dimension alterations (per dataset)
ds_list_CF = []
ensemble_vars = ["low","median","high"]

for i, j in zip(ds_list, list(itertools.chain(*all_files))):

    # extract values from xarray dataset and concatenate using numpy, then reshape to match dataset dimensions
    data_array = np.concatenate([i[var].values for var in ensemble_vars])
    data_array_r = data_array.reshape((len(i['low'].values),len(ensemble_vars)))
    
    # transform year data from integer to pd.time
    i = i.rename_vars({"years":"time"})
    i = i.set_coords(['time'])
    i = i.assign(time=('time', i['time'].values))
    
    # remove unconcatenated variables, to be filled in later using data_array_r
    i = i.drop(['index']+ensemble_vars)

    # expand dimension with data variables
    i = i.expand_dims(dim={"ensemble": np.array(ensemble_vars, dtype="S")})
 
    # add reshaped data array to the xarray dataset
    i = i.assign(slp=(["time", "ensemble"], data_array_r))

    # add or change certain variable / coordinate attributes
    dataset_attributes = {
        "slp": {"long_name": "sea level projections", "units": "m"},
        "time": {"long_name": "time projections", "units": "yr"},
        "ensemble": {"long_name": "ensemble", "units": "1"}, # set to 1 if no unit
    }  # specify custom (CF convention) attributes

    # add / overwrite attributes
    for k, v in dataset_attributes.items():
        try:
            i[k].attrs = dataset_attributes[k]
        except:
            continue

    #i.to_netcdf(path=str(j).replace(".csv", "_CF.nc")) # save single CF compliant files

    ds_list_CF.append(i)

In [25]:
# List all files that end with '_CF.nc'
CF_files = [f for f in os.listdir(os.path.join(dataset_dir,'pilot')) if f.endswith('_CF.nc')]

# Extract unique locations and climate projection scenario's (ssps)
locs = list(set([f.split('_')[1] for f in CF_files]))
ssps = list(set([f.split('_')[2] for f in CF_files]))

ds_list_CF_merge1 = []
for loc in locs:
    ds_ls = []
    for ds, CF_file in zip(ds_list_CF, CF_files):
        if loc in CF_file:
            ds_ls.append(ds)
    ds_list_CF_merge1.append(ds_ls)

In [28]:
ds_list_CF_merge2 = []
for ds_list, loc in zip(ds_list_CF_merge1, locs):
    ds = xr.concat(ds_list, dim='nssp')
    ds = ds.assign(ssp=(["nssp"], np.array(ssps, dtype="S")))
    ds = ds.set_coords("ssp")
    
    # add or change certain variable / coordinate attributes
    dataset_attributes = {"ssp": {"long_name": "shared socioeconomic pathways", "units": "1"} # set to 1 if no unit
        }  # specify custom (CF convention) attributes
    
    # add / overwrite attributes
    for k, v in dataset_attributes.items():
        try:
                ds[k].attrs = dataset_attributes[k]
        except:
                continue
    
    ds_list_CF_merge2.append(ds)

# merge areas
ds_CF_merge = xr.concat(ds_list_CF_merge2, dim="nlocs")
ds_CF_merge = ds_CF_merge.assign(locs=(["nlocs"], np.array(locs, dtype="S")))
ds_CF_merge = ds_CF_merge.set_coords("locs")

# Add coordinates of location (lat,lon)
lat = [43.5, 46, 44, 39.5]
lon = [-2, -1.5, -9, 0]

ds_CF_merge = ds_CF_merge.assign_coords(lat=("nlocs",lat))
ds_CF_merge = ds_CF_merge.assign_coords(lon=("nlocs",lon))

dataset_attributes = {"locs": {"long_name": "locations", "units": "1"},
                      "lat": {"long_name": "latitude", "units": "degrees"},
                      "lon": {"long_name": "longitude", "units": "degrees"} # set to 1 if no unit
        }  # specify custom (CF convention) attributes

for k, v in dataset_attributes.items():
        try:
                ds_CF_merge[k].attrs = dataset_attributes[k]
        except:
                continue

In [29]:
ds_CF_merge

<xarray.Dataset>
Dimensions:   (ensemble: 3, time: 4, nlocs: 4, nssp: 4)
Coordinates:
  * ensemble  (ensemble) object b'low' b'median' b'high'
  * time      (time) int64 2030 2050 2100 2150
    ssp       (nssp) |S6 b'high' b'ssp585' b'ssp245' b'ssp126'
    locs      (nlocs) |S12 b'SanSebastian' b'Valencia' b'Genova' b'LaRochelle'
    lat       (nlocs) float64 43.5 46.0 44.0 39.5
    lon       (nlocs) float64 -2.0 -1.5 -9.0 0.0
Dimensions without coordinates: nlocs, nssp
Data variables:
    slp       (nlocs, nssp, time, ensemble) float64 0.06823 0.1738 ... 1.908
Attributes: (12/21)
    TITLE:               Sea level projections
    TITLE_ABBREVIATION:  slp
    DESCRIPTION:         to be done later...
    SHORT_DESCRIPTION:   to be done later...
    INSTITUTION:         to be done later...
    PROVIDERS:           {"name": "Deltares Test", "url": "https://www.deltar...
    ...                  ...
    DOI:                 to be done later...
    LONG_NAME:           SLP
    UNITS:               m
    COMMENT:             test is in the works
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

In [30]:
# write to NetCDF file to check compliancy

# prevent file locking, see: https://github.com/pydata/xarray/issues/2376
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

ds_CF_merge.to_netcdf(path=str(dataset_dir_path).replace(".nc", "_CF.nc"))

In [31]:
%%capture cap --no-stderr

# check altered CF compliancy
check_compliancy(testfile=str(dataset_dir_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)

# save original CF compliancy (for first file)
save_compliancy(cap, testfile=str(dataset_dir_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)   

In [32]:
# export to zarr in write mode (to overwrite if exists)
ds_CF_merge.to_zarr(str(dataset_dir_path).replace(".nc", ".zarr"), mode="w")